In [1]:
! pip install bitsandbytes -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 12.9 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset, Dataset
import json

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `colab` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `colab`


In [127]:
sheet_path = 'Ka-ChatBot_BenchMark.xlsx'
faq_df = pd.read_excel(sheet_path, sheet_name="faq")
samples_df = pd.read_excel(sheet_path, sheet_name="samples")

faqs = faq_df['faq'].tolist()
ids = faq_df['idx'].tolist()
categories = faq_df['category'].tolist()

samples = samples_df['sample'].tolist()
labels = torch.tensor(samples_df['gt_idx'].tolist())

faq_list = zip(ids, faqs, categories)


In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,    # nested quantization (saves memory)
    bnb_4bit_quant_type="nf4",         # recommended quantization data type
    bnb_4bit_compute_dtype=torch.float16,  # computations in float16
)


tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    device_map=device,
    quantization_config=bnb_config,
)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [22]:
tokenizer.padding_side = 'left'
tokenizer.pad_token_id = 128001
model.generation_config.pad_token_id = tokenizer.pad_token_id

In [215]:
# best performance
instruction = """You are given a user question in Persian, a list of predefined questions with IDs, and the category each question belongs to. Your task is to find the single most semantically similar question.

Steps:
1. Determine which category the user question belongs to. Focus on meaning and intent.
2. Compare the user question only to questions within that category.
3. Choose the question whose intent or meaning best matches the user question.
"""
category_group = {}
for (id, q, cat) in zip(ids, faqs, categories):
  category_group[cat] = category_group.get(cat, []) + [(id, q)]
faq_list = "Predefined questions:\n"
for cat in category_group.keys():
  faq_list += f"\n\nCategory: {cat}\n" + '\n'.join(f"id: {id} - {q}" for id, q in category_group[cat])

rules = """Rules:
- All questions are in Persian. Do not translate, only compare.
- Return only a JSON object in the format: {"id": "<id>"}
- Do not include explanations or any other text."""


sysprompt = f"{instruction}\n\n{faq_list}\n\n{rules}"

In [188]:
instruction = """You are given a user question in Persian, a list of predefined questions with IDs, and the category each question belongs to. Your task is to find the single most semantically similar question."""
category_group = {}
for (id, q, cat) in zip(ids, faqs, categories):
  category_group[cat] = category_group.get(cat, []) + [(id, q)]
faq_list = "Predefined questions:\n"
for cat in category_group.keys():
  faq_list += f"\n\nCategory: {cat}\n" + '\n'.join(f"id: {id} - {q}" for id, q in category_group[cat])

rules = """

Steps:
0. All questions are in Persian. Do not translate, only compare.
1. Determine which category the user question belongs to. Focus on meaning and intent.
2. Compare the user question only to questions within that category.
3. Choose the question whose intent or meaning best matches the user question.
4. Return only a JSON object in the format: {"id": "<id>"}
5. Do not include explanations or any other text."""


sysprompt = f"{instruction}\n\n{faq_list}\n\n{rules}"

In [216]:
print(sysprompt)

You are given a user question in Persian, a list of predefined questions with IDs, and the category each question belongs to. Your task is to find the single most semantically similar question.

Steps:
1. Determine which category the user question belongs to. Focus on meaning and intent.
2. Compare the user question only to questions within that category.
3. Choose the question whose intent or meaning best matches the user question.


Predefined questions:


Category: احراز هویت
id: 1 - چرا من در احراز هویت افتتاح حساب خطای عدم تطبیق تصویر می گیرم؟ 

Category: افتتاح حساب
id: 2 - جگونه افتتاح حساب غیر حضوری کنم ؟
id: 3 - چگونه میتوانم حساب خودم را ببندم؟

Category: پرداخت قبض با حساب
id: 4 - چگونه پرداخت قبض با حساب/کارت انجام دهم؟

Category: تغییر تلفن همراه
id: 5 - چرا در زمان کارت به کارت اعلام می کند شماره در سامانه شاهکار ثبت نشده

Category: مسدودی کارت
id: 6 - چگونه کارت را از مسدودی در بیاورم؟
id: 7 - چگونه کارت خود را مسدود کنم؟ 

Category: تسهیلات در جریان و پرداخت قسط
id: 8 -

In [199]:
class QuestionDataset(Dataset):
  def __init__(self, questions, system_prompt):
    self.questions = questions
    self.system_prompt = system_prompt

  def __len__(self):
    return len(self.questions)

  def __getitem__(self, idx):
    messages = [
      {"role": "system", "content": self.system_prompt},
      {"role": "user", "content": self.questions[idx]},
    ]
    return messages


In [200]:
def collate_fn(batch):
  return tokenizer.apply_chat_template(
      batch, padding=True,
      add_generation_prompt=True,
      return_tensors='pt',
      return_dict=True).to(device)

In [201]:
userqs_ds = QuestionDataset(samples, sysprompt)
dataloader = DataLoader(userqs_ds, shuffle=False, batch_size=16, collate_fn=collate_fn)

In [202]:
allresponses = []
with torch.no_grad():
  for inputs in dataloader:
    outputs = model.generate(**inputs, max_new_tokens=50, do_sample=False)
    new_tokens = outputs[:, inputs["input_ids"].shape[-1]:-1]
    responses = tokenizer.batch_decode(new_tokens, skip_special_tokens=True)
    allresponses = allresponses + responses


In [203]:
pids = []
for resp in allresponses:
  try:
    pids.append(int(json.loads(resp)['id']))
  except:
    pids.append(0)
pids = torch.tensor(pids)

In [204]:
pids.count_nonzero()

tensor(92)

In [205]:
(pids == labels).sum() / len(labels)

tensor(0.7527)

In [212]:
resultdf = pd.DataFrame(
    {'samples': samples,
     'labels': labels,
     'llama3-8b-instruct': pids}
)

In [214]:
resultdf.to_csv('results.csv', index=False)